In [ ]:
# THỬ NGHIỆM TẤN CÔNG TOÀN DIỆN VỚI KẾT QUẢ CSV

print("🔬 BẮT ĐẦU THỬ NGHIỆM WATERMARKING TOÀN DIỆN")
print("="*70)

# Sử dụng ảnh đầu tiên làm demo
test_name, test_img = processed_images[0]
print(f"📸 Sử dụng ảnh: {test_name}")

# Nhúng watermark sử dụng DWT
print(f"🔧 Nhúng watermark với α = {config.DWT_ALPHA}")
watermarked_img = embed_watermark_dwt(
    test_img, 
    watermark, 
    alpha=config.DWT_ALPHA,
    wavelet=config.DWT_WAVELET, 
    level=config.DWT_LEVEL
)

# Tính PSNR để đánh giá chất lượng
psnr_value = calculate_psnr(test_img, watermarked_img)

print(f"✅ Nhúng watermark hoàn tất")
print(f"📊 PSNR: {psnr_value:.2f} dB")

# Đánh giá chất lượng nhúng
print(f"\n📈 ĐÁNH GIÁ CHẤT LƯỢNG NHÚNG:")
print(f"  🎯 PSNR: {psnr_value:.2f} dB")
if psnr_value >= 30:
    print("  ✅ Chất lượng tốt (PSNR ≥ 30 dB)")
elif psnr_value >= 20:
    print("  ⚠️ Chất lượng chấp nhận được (20-30 dB)")
else:
    print("  ❌ Chất lượng thấp (PSNR < 20 dB)")

print(f"  🔸 Cường độ nhúng α: {config.DWT_ALPHA}")
print(f"  🌊 Wavelet: {config.DWT_WAVELET}, level: {config.DWT_LEVEL}")

# Hiển thị kết quả nhúng
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Ảnh gốc
axes[0,0].imshow(test_img)
axes[0,0].set_title(f'Ảnh gốc\n{test_img.shape}')
axes[0,0].axis('off')

# Ảnh đã nhúng watermark
axes[0,1].imshow(watermarked_img)
axes[0,1].set_title(f'Ảnh đã nhúng watermark\nPSNR: {psnr_value:.2f} dB')
axes[0,1].axis('off')

# Watermark được nhúng
axes[1,0].imshow(watermark, cmap='gray')
axes[1,0].set_title(f'Watermark nhị phân\n{watermark.shape}')
axes[1,0].axis('off')

# Sự khác biệt (được phóng đại)
diff_img = np.abs(test_img.astype(np.float32) - watermarked_img.astype(np.float32))
diff_enhanced = np.clip(diff_img * 3, 0, 255).astype(np.uint8)
axes[1,1].imshow(diff_enhanced)
axes[1,1].set_title('Sự khác biệt (×3)')
axes[1,1].axis('off')

plt.suptitle('Kết quả nhúng watermark bằng DWT', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# THỰC HIỆN TẤN CÔNG TOÀN DIỆN
print(f"\n⚔️ BẮT ĐẦU THỬ NGHIỆM TẤN CÔNG HÌNH HỌC")
print("="*70)

# Danh sách tất cả các tấn công
all_attacks = list(GEOMETRIC_ATTACKS.keys())
print(f"📋 Sẽ thực hiện {len(all_attacks)} loại tấn công:")
for i, attack in enumerate(all_attacks, 1):
    print(f"   {i}. {attack}")

# Khởi tạo danh sách lưu kết quả cho CSV
attack_results = []

# Thực hiện từng loại tấn công
print(f"\n🔄 TIẾN HÀNH THỬ NGHIỆM:")
for i, (attack_name, attack_func) in enumerate(GEOMETRIC_ATTACKS.items(), 1):
    print(f"\n🎯 [{i}/{len(all_attacks)}] Tấn công: {attack_name}")
    
    try:
        # Áp dụng tấn công lên ảnh đã nhúng watermark
        attacked_img = attack_func(watermarked_img)
        print(f"   ✅ Áp dụng tấn công thành công")
        
        # Trích xuất watermark bằng 3 phương pháp
        print("   🔍 Trích xuất watermark bằng 3 phương pháp...")
        
        # 1. Phương pháp đề xuất: DWT + SIFT
        wm_extracted_robust = extract_watermark_robust(
            attacked_img, test_img, config.WATERMARK_SIZE,
            alpha=config.DWT_ALPHA, wavelet=config.DWT_WAVELET, level=config.DWT_LEVEL
        )
        
        # 2. DWT đơn thuần (không SIFT)
        wm_extracted_dwt_only = extract_watermark_dwt_only(
            attacked_img, config.WATERMARK_SIZE,
            wavelet=config.DWT_WAVELET, level=config.DWT_LEVEL
        )
        
        # 3. SIFT đơn thuần (không DWT tham chiếu)
        wm_extracted_sift_only = extract_watermark_sift_only(
            attacked_img, test_img, config.WATERMARK_SIZE
        )
        
        # Tính toán metrics
        ncc_robust = calculate_ncc(watermark, wm_extracted_robust)
        ber_robust = calculate_ber(watermark, wm_extracted_robust)
        ncc_dwt = calculate_ncc(watermark, wm_extracted_dwt_only)
        ber_dwt = calculate_ber(watermark, wm_extracted_dwt_only)
        ncc_sift = calculate_ncc(watermark, wm_extracted_sift_only)
        ber_sift = calculate_ber(watermark, wm_extracted_sift_only)
        
        # Lưu kết quả cho CSV
        results = {
            'Attack_Type': attack_name,
            'DWT+SIFT_NCC': round(ncc_robust, 4),
            'DWT+SIFT_BER': round(ber_robust, 4),
            'DWT_only_NCC': round(ncc_dwt, 4),
            'DWT_only_BER': round(ber_dwt, 4),
            'SIFT_only_NCC': round(ncc_sift, 4),
            'SIFT_only_BER': round(ber_sift, 4),
            'PSNR_Embedding': round(psnr_value, 2)
        }
        
        attack_results.append(results)
        
        # Hiển thị kết quả ngắn gọn
        print(f"   📊 DWT+SIFT: NCC={ncc_robust:.3f}, BER={ber_robust:.3f}")
        print(f"      DWT-only: NCC={ncc_dwt:.3f}, BER={ber_dwt:.3f}")
        print(f"      SIFT-only: NCC={ncc_sift:.3f}, BER={ber_sift:.3f}")
        
    except Exception as e:
        print(f"   ❌ Lỗi trong quá trình trích xuất: {e}")
        # Thêm kết quả thất bại vào CSV
        results = {
            'Attack_Type': attack_name,
            'DWT+SIFT_NCC': 0.0000,
            'DWT+SIFT_BER': 1.0000,
            'DWT_only_NCC': 0.0000,
            'DWT_only_BER': 1.0000,
            'SIFT_only_NCC': 0.0000,
            'SIFT_only_BER': 1.0000,
            'PSNR_Embedding': round(psnr_value, 2)
        }
        attack_results.append(results)

# Tạo DataFrame từ kết quả
results_df = pd.DataFrame(attack_results)

print(f"\n📊 TỔNG HỢP KẾT QUẢ THỬ NGHIỆM")
print("="*80)

# Hiển thị bảng CSV đẹp
print("\n📋 BẢNG KẾT QUẢ CHI TIẾT (CSV FORMAT):")
csv_content = results_df.to_csv(index=False)
print(csv_content)

# Lưu ra file CSV
import datetime
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
csv_filename = f"watermarking_results_{config.DWT_WAVELET}_alpha{config.DWT_ALPHA}_{timestamp}.csv"
results_df.to_csv(csv_filename, index=False)
print(f"💾 Đã lưu kết quả vào file: {csv_filename}")

# Thống kê tổng quan
print(f"\n📈 THỐNG KÊ HIỆU SUẤT TRUNG BÌNH:")
print("="*50)

summary_stats = {
    'Method': ['DWT+SIFT (Proposed)', 'DWT-only', 'SIFT-only'],
    'Avg_NCC': [
        results_df['DWT+SIFT_NCC'].mean(),
        results_df['DWT_only_NCC'].mean(), 
        results_df['SIFT_only_NCC'].mean()
    ],
    'Avg_BER': [
        results_df['DWT+SIFT_BER'].mean(),
        results_df['DWT_only_BER'].mean(),
        results_df['SIFT_only_BER'].mean()
    ],
    'Best_NCC': [
        results_df['DWT+SIFT_NCC'].max(),
        results_df['DWT_only_NCC'].max(),
        results_df['SIFT_only_NCC'].max()
    ]
}

summary_df = pd.DataFrame(summary_stats)
print("\n📊 BẢNG TỔNG HỢP HIỆU SUẤT:")
print(summary_df.round(4).to_string(index=False))

# Tìm phương pháp tốt nhất cho từng tấn công
print(f"\n🏆 PHƯƠNG PHÁP TỐT NHẤT CHO TỪNG TẤN CÔNG (theo NCC):")
print("="*60)
for _, row in results_df.iterrows():
    ncc_values = [row['DWT+SIFT_NCC'], row['DWT_only_NCC'], row['SIFT_only_NCC']]
    methods = ['DWT+SIFT', 'DWT-only', 'SIFT-only']
    best_idx = np.argmax(ncc_values)
    best_method = methods[best_idx]
    best_ncc = ncc_values[best_idx]
    print(f"  {row['Attack_Type']:15} → {best_method:10} (NCC: {best_ncc:.4f})")

# Lưu summary stats
summary_filename = f"watermarking_summary_{timestamp}.csv"
summary_df.to_csv(summary_filename, index=False)
print(f"\n💾 Đã lưu bảng tổng hợp vào: {summary_filename}")

print(f"\n✅ HOÀN THÀNH THỬ NGHIỆM TẤN CÔNG!")
print("="*70)
